# Example using Geography Module

FAModel was originally designed to represent a floating array model in arbitrary space but it also has the ability to represent a model in an actual, global space (e.g., a specific floating offshore wind lease area).

This file will walk through the options available to a user who wishes to use geography-related design inputs

### CRS Setup

The first step towards using geography-related tools is to set the proper coordindate-reference system (CRS). There are many CRSs that set up coordinates for the world in either a geographic coordinate system (i.e., latitude and longitude) or a projected coordinate system (i.e., UTM). The main goal is to represent the 3-D coordinates of a globe into a 2-D plane. Each coordinate reference system is going to distort the global map in some way and you're not going to be able to preserve all properties (like distances or areas). That's why Greenland appears so big on some maps. Imagine peeling an orange and then laying the peels out on a table into a rectangle...it's hard to do.

There is a Python package called 'pyproj' which we use to store CRS information. As a start, we can set a 'global' CRS based on the conventional lat/long coordinate system, which is signified by a specific EPSG code: 4326

In [112]:
from pyproj import CRS

latlong_crs = CRS.from_epsg(4326)

We can also create other CRSs based on our interests. We can create a UTM CRS (a common 2D projection) if we have a specific location we want to focus on. (Think of UTM CRSs as if you cut the Earth into 60 equidistance slices from the north pole to just above the equator and from the south pole to just below the equator). If you have a specific area of interest, the UTM projection will represent that area very well, but not exactly.

In [113]:
from pyproj.aoi import AreaOfInterest
from pyproj.database import query_utm_crs_info
import famodel.geography as geo

target_crs = geo.getTargetCRS([-125, -124], [40, 41])

More appropriately, we can create 'custom' CRSs that create a reference system about a specific longitude and latitude (i.e., the centroid of an offshore wind lease area). This will result in the least amount of distortion in the results.

In [114]:
custom_crs = geo.getCustomCRS(-124.5, 40.5)

### Boundaries

Next, we can set up specific boundaries in space (like a lease area, or any area that can be defined by latitude and longitude coordinates). We will use the Humboldt offshore wind lease area as an example.

We use the Python package 'geopandas' to store geography-related information, in the conventional pandas format (the main difference between pandas and geopandas is that geopandas uses the 'shapely' python package to store coordinates and shapes).

We can create a geopandas dataframe using 'shape files' (.shp) to store the boundary information contained in the shape file,

In [115]:
import geopandas as gpd

lease_areas = gpd.read_file('../geography/Wind_Lease_Outlines_2_2023.shp')

and then we can extract certain lease area information from that geopandas dataframe (for futher development, you will have to take a peek inside the geodataframe to determine what exact part of the shapefile you want to extract).

In [116]:
lease_area = lease_areas.loc[lease_areas['LEASE_NUMB']=='OCS-P0562 - Provisional']

You can then extract the latitudes and longitudes of the boundary of that shape,

In [117]:
area_longs, area_lats = lease_area.geometry.unary_union.exterior.coords.xy

or even get the centroid of the lease area.

In [118]:
centroid = ( lease_area.geometry.centroid.values.x[0], lease_area.geometry.centroid.values.y[0] )

C:\Users\shousner\AppData\Local\Temp\1\ipykernel_23620\3218981639.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = ( lease_area.geometry.centroid.values.x[0], lease_area.geometry.centroid.values.y[0] )


All of this is done in the geography module's function ```getLeaseCoords(lease_name)```.

Next, we need to convert these latitudes and longitudes to meters, since that is what FAModel is most familiar with. This is where the custom CRS is useful. We can convert the latitudes and longitudes into meters relative to a specific reference point. We run the following converter function to get the boundaries in units of meters.

This function uses the geopandas.to_crs() function to convert the data and then ensure that the new coordinates are relative to the input centroid (the input centroid should be the same point used to create the custom CRS).

This process can also be reversed using the 'convertMeters2LatLong()' function

In [119]:
lease_name = 'Humboldt_SW'
lease_longs, lease_lats, centroid_latlong = geo.getLeaseCoords(lease_name)
lease_xs, lease_ys, centroid_m = geo.convertLatLong2Meters(lease_longs, lease_lats, centroid_latlong, latlong_crs, custom_crs, return_centroid=True)

c:\code\floatingarraydesign\famodel\famodel\geography.py:146: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = ( lease_area.geometry.centroid.values.x[0], lease_area.geometry.centroid.values.y[0] )


### Bathymetry

Once the boundaries are set up, the next important part is to set up the bathymetry based on an input GEBCO file. This can all be done by running something like the following 

In [120]:
gebco_file = '../geography/gebco_humboldt_2023_n41.3196_s40.3857_w-125.2881_e-123.9642.asc'
bath_longs, bath_lats, bath_depths, ncols, nrows = geo.getMapBathymetry(gebco_file)
bath_xs, bath_ys, bath_depths = geo.convertBathymetry2Meters(bath_longs, bath_lats, bath_depths, centroid_latlong, centroid_m, latlong_crs, custom_crs, ncols, nrows)

This section reads in a GEBCO file and extracts a list of latitudes and longitudes that define a "rectangle" of coordinates, where each point within that rectangle has a depth. However, in the 'meters' reference system, this will be a slightly distorted rectangle due to the differences in CRSs. This function also returns the discretization in the number of rows and columns.

Those latitude and longitude values are then read into the next function, but only the maximum and minimum values are used to generate a perfect rectangle in the custom_CRS "inside" of the distorted geographic rectangle. It can set a new discretization between those maximum and minimum points, creates a python meshgrid, and then converts this new square back into the geographic coordinate system to make a slightly smaller distorted rectangle. It only does this so that it can reference the geographic coordinate system's depths and assign the right depths to the new "custom CRS" square (using the getDepthFromBathymetry function). The result is a list of x coordinates (2D array), y coordinates (2D array), and water depths (3D array) at each x/y point.

For use with FAModel/MoorPy, you can then write this information to a MoorPy/MoorDyn-readable text file:

In [121]:
geo.writeBathymetryFile('bathymetry_humboldt.txt', bath_xs, bath_ys, bath_depths)

All of the above processes can be summarized in:

In [122]:
info = geo.getLeaseAndBathymetryInfo(lease_name, gebco_file, bath_ncols=100, bath_nrows=100)
print(info.keys())

c:\code\floatingarraydesign\famodel\famodel\geography.py:146: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid = ( lease_area.geometry.centroid.values.x[0], lease_area.geometry.centroid.values.y[0] )


dict_keys(['lease_longs', 'lease_lats', 'lease_centroid', 'lease_xs', 'lease_ys', 'bath_longs', 'bath_lats', 'bath_xs', 'bath_ys', 'bath_depths'])


### Soil

Similar processes can be done for extracting and setting up soil information

```getSoilGrid()``` reads an input soil shape file and creates a rectangular grid of x coordinates (2D), y coordinates (2D) and soil names (3D string array), based on the shape file while also converting lat/long information to meters. It works by creating shape objects based on the data of the shapefile and using data within the shape file to extract the soil type name (e.g., 'mud', or 'rock')

```getSoilType()``` returns the soil type name (string) that an input lat/long is above

These are just the immediate needs for the related floating array modeling and design work. There are likely many more applications using geography-related tools that can be done.